# Quickstart: Pandas API on Spark

This is a short introduction to pandas API on Spark, geared mainly for new users. This notebook shows you some key differences between pandas and pandas API on Spark. You can run this examples by yourself in 'Live Notebook: pandas API on Spark' at [the quickstart page](https://spark.apache.org/docs/latest/api/python/getting_started/index.html).

Customarily, we import pandas API on Spark as follows:

In [1]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

/usr/local/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


## Object Creation



Creating a pandas-on-Spark Series by passing a list of values, letting pandas API on Spark create a default integer index:

In [2]:
s = ps.Series([1, 3, 5, np.nan, 6, 8])

In [3]:
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

Creating a pandas-on-Spark DataFrame by passing a dict of objects that can be converted to series-like.

In [4]:
psdf = ps.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

In [5]:
psdf

,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


Creating a pandas DataFrame by passing a numpy array, with a datetime index and labeled columns:

In [6]:
#dates = pd.date_range('20130101', periods=6)
dates = [10, 20, 30, 40, 50, 60]

In [7]:
dates

[10, 20, 30, 40, 50, 60]

In [8]:
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))

In [9]:
pdf

,A,B,C,D
10,0.433842,0.535376,-0.108796,0.893840
20,1.243993,-0.130474,0.055239,0.425580
30,-0.663595,-0.474558,0.188291,-0.370572
40,0.128893,1.426027,0.921554,0.527648
50,-1.328009,-0.324521,-0.292307,-1.039993
60,1.575341,-0.398749,-1.483577,1.676350


Now, this pandas DataFrame can be converted to a pandas-on-Spark DataFrame

In [10]:
psdf = ps.from_pandas(pdf)

In [11]:
type(psdf)

pyspark.pandas.frame.DataFrame

It looks and behaves the same as a pandas DataFrame.

In [12]:
psdf

,A,B,C,D
10,0.433842,0.535376,-0.108796,0.893840
20,1.243993,-0.130474,0.055239,0.425580
30,-0.663595,-0.474558,0.188291,-0.370572
40,0.128893,1.426027,0.921554,0.527648
50,-1.328009,-0.324521,-0.292307,-1.039993
60,1.575341,-0.398749,-1.483577,1.676350


Also, it is possible to create a pandas-on-Spark DataFrame from Spark DataFrame easily.

Creating a Spark DataFrame from pandas DataFrame

In [13]:
spark = SparkSession.builder.getOrCreate()

In [14]:
sdf = spark.createDataFrame(pdf)

In [15]:
sdf.show()

+-------------------+--------------------+--------------------+-------------------+
|                  A|                   B|                   C|                  D|
+-------------------+--------------------+--------------------+-------------------+
| 0.4338424599047517|   0.535376266524015|-0.10879636216829777| 0.8938398579298743|
| 1.2439929664066671|-0.13047374137002218|0.055238975541590914|0.42558030958040194|
|-0.6635947967208623| -0.4745579145773351|  0.1882914096997109|-0.3705717258998855|
|0.12889340545311015|  1.4260265452309562|  0.9215539586951889| 0.5276476807161806|
|-1.3280093861278222| -0.3245206012359708| -0.2923071882975252|-1.0399927526101116|
| 1.5753408893174288| -0.3987485830922929| -1.4835767279668848|  1.676349653079628|
+-------------------+--------------------+--------------------+-------------------+



Creating pandas-on-Spark DataFrame from Spark DataFrame.

In [16]:
psdf = sdf.pandas_api()

In [17]:
psdf

,A,B,C,D
0,0.433842,0.535376,-0.108796,0.893840
1,1.243993,-0.130474,0.055239,0.425580
2,-0.663595,-0.474558,0.188291,-0.370572
3,0.128893,1.426027,0.921554,0.527648
4,-1.328009,-0.324521,-0.292307,-1.039993
5,1.575341,-0.398749,-1.483577,1.676350


Having specific [dtypes](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-dtypes) . Types that are common to both Spark and pandas are currently supported.

In [18]:
psdf.dtypes

A    float64
B    float64
C    float64
D    float64
dtype: object

Here is how to show top rows from the frame below.

Note that the data in a Spark dataframe does not preserve the natural order by default. The natural order can be preserved by setting `compute.ordered_head` option but it causes a performance overhead with sorting internally.

In [19]:
psdf.head()

,A,B,C,D
0,0.433842,0.535376,-0.108796,0.893840
1,1.243993,-0.130474,0.055239,0.425580
2,-0.663595,-0.474558,0.188291,-0.370572
3,0.128893,1.426027,0.921554,0.527648
4,-1.328009,-0.324521,-0.292307,-1.039993


Displaying the index, columns, and the underlying numpy data.

In [20]:
psdf.index

Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [21]:
psdf.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [22]:
psdf.to_numpy()

/usr/local/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


array([[ 0.43384246,  0.53537627, -0.10879636,  0.89383986],
       [ 1.24399297, -0.13047374,  0.05523898,  0.42558031],
       [-0.6635948 , -0.47455791,  0.18829141, -0.37057173],
       [ 0.12889341,  1.42602655,  0.92155396,  0.52764768],
       [-1.32800939, -0.3245206 , -0.29230719, -1.03999275],
       [ 1.57534089, -0.39874858, -1.48357673,  1.67634965]])

Showing a quick statistic summary of your data

In [23]:
psdf.describe()

,A,B,C,D
count,6.000000,6.000000,6.000000,6.000000
mean,0.231744,0.105517,-0.119933,0.352142
std,1.105964,0.742956,0.787368,0.953259
min,-1.328009,-0.474558,-1.483577,-1.039993
25%,-0.663595,-0.398749,-0.292307,-0.370572
50%,0.128893,-0.324521,-0.108796,0.425580
75%,1.243993,0.535376,0.188291,0.893840
max,1.575341,1.426027,0.921554,1.676350


Transposing your data

In [24]:
psdf.T

,0,1,2,3,4,5
A,0.433842,1.243993,-0.663595,0.128893,-1.328009,1.575341
B,0.535376,-0.130474,-0.474558,1.426027,-0.324521,-0.398749
C,-0.108796,0.055239,0.188291,0.921554,-0.292307,-1.483577
D,0.893840,0.425580,-0.370572,0.527648,-1.039993,1.676350


Sorting by its index

In [25]:
psdf.sort_index(ascending=False)

,A,B,C,D
5,1.575341,-0.398749,-1.483577,1.676350
4,-1.328009,-0.324521,-0.292307,-1.039993
3,0.128893,1.426027,0.921554,0.527648
2,-0.663595,-0.474558,0.188291,-0.370572
1,1.243993,-0.130474,0.055239,0.425580
0,0.433842,0.535376,-0.108796,0.893840


Sorting by value

In [26]:
psdf.sort_values(by='B')

,A,B,C,D
2,-0.663595,-0.474558,0.188291,-0.370572
5,1.575341,-0.398749,-1.483577,1.676350
4,-1.328009,-0.324521,-0.292307,-1.039993
1,1.243993,-0.130474,0.055239,0.425580
0,0.433842,0.535376,-0.108796,0.893840
3,0.128893,1.426027,0.921554,0.527648


## Missing Data
Pandas API on Spark primarily uses the value `np.nan` to represent missing data. It is by default not included in computations. 


In [27]:
pdf1 = pdf.reindex(index=dates[0:4], columns=list(pdf.columns) + ['E'])

In [28]:
pdf1.loc[dates[0]:dates[1], 'E'] = 1

In [29]:
psdf1 = ps.from_pandas(pdf1)

In [30]:
psdf1

,A,B,C,D,E
10,0.433842,0.535376,-0.108796,0.893840,1.0
20,1.243993,-0.130474,0.055239,0.425580,1.0
30,-0.663595,-0.474558,0.188291,-0.370572,NaN
40,0.128893,1.426027,0.921554,0.527648,NaN


To drop any rows that have missing data.

In [31]:
psdf1.dropna(how='any')

,A,B,C,D,E
10,0.433842,0.535376,-0.108796,0.89384,1.0
20,1.243993,-0.130474,0.055239,0.42558,1.0


Filling missing data.

In [32]:
psdf1.fillna(value=5)

,A,B,C,D,E
10,0.433842,0.535376,-0.108796,0.893840,1.0
20,1.243993,-0.130474,0.055239,0.425580,1.0
30,-0.663595,-0.474558,0.188291,-0.370572,5.0
40,0.128893,1.426027,0.921554,0.527648,5.0


## Operations

### Stats
Performing a descriptive statistic:

In [33]:
psdf.mean()

A    0.231744
B    0.105517
C   -0.119933
D    0.352142
dtype: float64

### Spark Configurations

Various configurations in PySpark could be applied internally in pandas API on Spark.
For example, you can enable Arrow optimization to hugely speed up internal pandas conversion. See also <a href="https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html">PySpark Usage Guide for Pandas with Apache Arrow</a> in PySpark documentation.

In [34]:
prev = spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")  # Keep its default value.
ps.set_option("compute.default_index_type", "distributed")  # Use default index prevent overhead.
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings coming from Arrow optimizations.

In [35]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
%timeit ps.range(300000).to_pandas()

184 ms ± 57.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", False)
%timeit ps.range(300000).to_pandas()

1.22 s ± 57.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
ps.reset_option("compute.default_index_type")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", prev)  # Set its default value back.

## Grouping
By “group by” we are referring to a process involving one or more of the following steps:

- Splitting the data into groups based on some criteria
- Applying a function to each group independently
- Combining the results into a data structure

In [38]:
psdf = ps.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B': ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                    'C': np.random.randn(8),
                    'D': np.random.randn(8)})

In [39]:
psdf

,A,B,C,D
0,foo,one,-1.382838,-0.590871
1,bar,one,0.093406,1.157414
2,foo,two,-0.972455,-1.067182
3,bar,three,-1.222598,0.885407
4,foo,two,0.299365,-0.292654
5,bar,two,-0.321021,-1.819620
6,foo,one,0.091621,0.498738
7,foo,three,-1.008889,0.123778


Grouping and then applying the [sum()](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.groupby.GroupBy.sum.html) function to the resulting groups.

In [40]:
psdf.groupby('A').sum()

,C,D
A,,
foo,-2.973196,-1.328191
bar,-1.450213,0.223201


Grouping by multiple columns forms a hierarchical index, and again we can apply the sum function.

In [41]:
psdf.groupby(['A', 'B']).sum()

C         D
A   B                        
foo one   -1.291217 -0.092133
bar one    0.093406  1.157414
foo two   -0.673090 -1.359837
bar three -1.222598  0.885407
    two   -0.321021 -1.819620
foo three -1.008889  0.123778

## Plotting

In [42]:
pser = pd.Series(np.random.randn(1000),
                 index=pd.date_range('1/1/2000', periods=1000))

In [43]:
psser = ps.Series(pser)

In [44]:
psser = psser.cummax()

In [45]:
psser.plot()

ImportError: plotly is required for plotting when the default backend 'plotly' is selected.

On a DataFrame, the [plot()](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.plot.html) method is a convenience to plot all of the columns with labels:

In [ ]:
pdf = pd.DataFrame(np.random.randn(1000, 4), index=pser.index,
                   columns=['A', 'B', 'C', 'D'])

In [ ]:
psdf = ps.from_pandas(pdf)

In [ ]:
psdf = psdf.cummax()

In [ ]:
psdf.plot()

For more details, [Plotting](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/frame.html#plotting) documentation.

## Getting data in/out

### CSV

CSV is straightforward and easy to use. See [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.to_csv.html) to write a CSV file and [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.read_csv.html) to read a CSV file.

In [ ]:
psdf.to_csv('foo.csv')
ps.read_csv('foo.csv').head(10)

### Parquet

Parquet is an efficient and compact file format to read and write faster. See [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.to_parquet.html) to write a Parquet file and [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.read_parquet.html) to read a Parquet file.

In [ ]:
psdf.to_parquet('bar.parquet')
ps.read_parquet('bar.parquet').head(10)

### Spark IO

In addition, pandas API on Spark fully supports Spark's various datasources such as ORC and an external datasource.  See [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.to_orc.html) to write it to the specified datasource and [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.read_orc.html) to read it from the datasource.

In [ ]:
psdf.to_spark_io('zoo.orc', format="orc")
ps.read_spark_io('zoo.orc', format="orc").head(10)

See the [Input/Output](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/io.html) documentation for more details.